<a href="https://colab.research.google.com/github/IgorFelipi/Analise-de-dados-Bear-Atacks-/blob/main/Bear_Atacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('data (3).csv')

**TRABALHANDO A PLANILHA, LIMPANDO E ATUALIZANDO OS DEVIDOS TIPOS DE DADOS**

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

In [36]:
df.dtypes
df[' age'] = df[' age'].astype(int)

In [ ]:
df['Date']= pd.to_datetime(df['Date'])

In [ ]:
df.head()

**FAZENDO A ANALISE DE ATAQUES POR ANO E GENERO**

In [ ]:
# @title Qual gênero sofre mais ataques.
# Com isso criamos uma variavel de ataque por ano e vemos que o maior fora 2018 com 6 no total

atacks_per_year = df.groupby(['Year', 'gender']).size().reset_index(name='count')

gender_total = atacks_per_year.groupby('Year')['count'].sum().reset_index()

#gender_total.describe()

# Neste ponto usamos novamente para saber qual o genero mais atacado, male or female
Gender_count = atacks_per_year.groupby('gender')['count'].sum().reset_index(name='total_atacks')

total_atacks_overall =  Gender_count['total_atacks'].sum()

# Por fim temos a porcentagem e com isso concluimos que os homens sofrem 46% a mais ataques que as mulheres
Gender_count['percentage'] = (Gender_count['total_atacks'] / total_atacks_overall) * 100
Gender_count = Gender_count.sort_values(by='percentage', ascending=False)
display(Gender_count)



TIPO DE URSO POR LOCALIZAÇÃO, AONDE TEM MAIS ATAQUES

In [ ]:
# @title Local de mais ataques
atacks_by_location = df.groupby('Location').size().reset_index(name='total_atacks')
atacks_by_location = atacks_by_location.sort_values(by='total_atacks', ascending=False)
most_atacks_location = atacks_by_location.iloc[0]
display(most_atacks_location)

df_most_atacks_location = df[df['Location'] == most_atacks_location['Location']]

atacks_by_gender = df_most_atacks_location.groupby('gender').size().reset_index(name='total_atacks')
atacks_by_gender = atacks_by_gender.sort_values(by = 'total_atacks', ascending=False)
display(atacks_by_gender)

Foi visto que neste código o lugar com mais ataques foi **Glacier National Park Montana** com um total de **10**.

In [ ]:
# @title Urso mais perigoso (mais ataques).
Dangerous_bear = df.groupby('Type of bear').size().reset_index(name='Number of Atacks')
Dangerous_bear = Dangerous_bear.sort_values(by='Number of Atacks', ascending=False)
most_dangerous_bear = Dangerous_bear.iloc[0]


df_most_bear_atacks = df[df['Type of bear'] == most_dangerous_bear['Type of bear']]

atacks_by_ecosystem = df_most_bear_atacks.groupby(['Type', 'Type of bear', 'Year']).size().reset_index(name='total_atacks')
atacks_by_ecosystem = atacks_by_ecosystem.sort_values(by='total_atacks', ascending=False)
display(atacks_by_ecosystem)

Neste trecho acima concluimos que o urso mais perigoso é o **Brown Bear** e o local que mais atacou juntamente com o ano foi na **Wild** em **2018** com um total de 5 ataques.

In [ ]:
# @title Total de Mortes e sobreviventes.
# Verificando o total de 1 = obito e 0 = sobreviveu
People_killed = df.groupby('Only one killed').size().reset_index(name= 'Total')
People_killed = People_killed.sort_values(by='Total', ascending=False)

filter = "Wild or captive"
filter_df = df[df['Type'] == filter]

# o plt serve para construir o gráfico desejado de porcentagem de sobreviventes
plt.figure(figsize=(8,8))
plt.pie(People_killed['Total'], labels = People_killed['Only one killed'], autopct= '%1.1f%%', startangle=90)
plt.axis('equal')
plt.title('Percentage of people killed = 1 or survided = 0')
plt.show()


VIMOS QUE O TOTAL DE SOBREVIVENTES FOI INFERIOR AO TOTAL DE MORTES, REGISTRANDO UM ALTO INDICE DE SE TORNAR UMA VITIMA FATAL DESTES ATAQUES.

In [ ]:
# @title Proporção de sobrevivência
killed_and_type = df.groupby(['Type', 'Only one killed']).size().reset_index(name='total')

labels = [f"{row['Type']} - {'Killed' if row['Only one killed'] == 1 else 'Survived'}" for _,row in killed_and_type.iterrows()]
sizes = killed_and_type['total']

plt.figure(figsize=(8,8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title('Percentage of people killed or survided by wild and Captive')
plt.axis('equal')
plt.show()

COM ESTE GRÁFICO VIMOS QUE A CHANCE DE SOBREVIVER ESTANDO NA FLORESTA (EM QUESTÃO DE PROPROCIONALIDADE) É MAIOR DO QUE ESTANDO EM CATIVEIRO.

# º Gerar uma analise com uma LLMs GenAI para esclarecer as descrições dos ataques, qual motivo mais recorrente que possa ocasionar um ataque as humanos.

In [ ]:
# @title IMPORTANDO O GOOGLE GEMINI E USANDO A API_KEY
# importando e instalando o google gemini
import google.generativeai as genai
from google.colab import userdata
Api_key = 'SUA GEMINI API AQUI' # @param {type:"string"}
genai.configure(api_key= Api_key)

model = genai.GenerativeModel('gemini-pro')

In [ ]:
# @title FAZENDO A DEVIDA ANALISE COM A AJUDA DA API
# Fazendo o código para analisar o comportamento das descrições

def analisar_descricoes(df):
  lista = df['Description'].tolist()
  sistema = "Você é um modelo de análise de texto, para descrever os motivos que levaram ao ataque do urso." #@param {type: "string"}
  prompt = f"""
    {sistema}

    Sintetize uma análise geral sobre a lista de descrições fornecidas:
    {lista}
    """

     # Gerar resposta
  resposta = model.generate_content(prompt)

  return resposta.text

# Usar a função
insights = analisar_descricoes(df)
print(insights)